In [6]:
from bkz import bkz_bit_security, estimate_bkz_security_for_m, PESSIMISTIC, HEURISTIC, SVP_ORACLES, log2gamma
from combinatorials import cpw_bit_security, cpw_bit_security_pessimistic, estimate_cpw_with_hnf_and_parameters, estimate_cpw_with_parameters
import math
from bkz2 import estimate_bkz as estimate_bkz2
import direct_svp


old_params = [\
    ["A1", 64, 5, 32],\
    ["A2", 64, 7, 64],\
    ["A3", 64, 11, 128],\
    ["A4", 64, 16, 256],\
    ["A5", 64, 22, 512],\
    ["A6", 64, 32, 1024],\
    ["B1", 254, 3, 2],\
    ["B2", 254, 4, 4],\
    ["B3", 254, 5, 8],\
    ["B4", 254, 7, 16],\
    ["B5", 254, 11, 32],\
    ["B6", 254, 16, 64],\
    ["B7", 254, 24, 128],\
    ["B8", 254, 32, 256],\
    ["B9", 254, 51, 512],\
]

cur_params = [\
    ["A1", 64, 2, 32],\
    ["A2", 64, 4, 64],\
    ["A3", 64, 6, 128],\
    ["A4", 64, 10, 256],\
    ["A5", 64, 16, 512],\
    ["A6", 64, 22, 1024],\
    ["B3", 254, 1, 3],
    ["B3", 254, 2, 7],
    ["B4", 254, 4, 16],\
    ["B5", 254, 6, 32],\
    ["B6", 254, 10, 64],\
    ["B7", 254, 16, 128],\
    ["B8", 254, 24, 256],\
    ["B9", 254, 32, 512],\
]

future_params = [\
    ["A1", 256, 1, 2],
    ["A1", 256, 2, 4],
    ["A1", 256, 4, 7],
    ["A1", 256, 8, 18],
    ["A1", 256, 16, 54],
    ["A1", 256, 24, 112],
    ["A1", 256, 32, 192],
    ["G1", 64, 1, 19],
    ["G1", 64, 2, 20],
    ["G1", 64, 4, 32],
    ["G1", 64, 8, 64],
    ["G1", 64, 16, 216],
    ["G1", 64, 24, 448],
    ["G1", 64, 32, 768], 
]

def estimate_set_of_params(params):
    print("|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|".format("$log_2(q)$", "$log_2(beta)$", "$n$", "SVP (L2)", "SVP (Loo)", "BKZ attack", "CPW attack", "#limbs", "pi"))
    eighteenDash = "-" * 18
    print("|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|".format(eighteenDash, eighteenDash, eighteenDash, eighteenDash, eighteenDash, eighteenDash, eighteenDash, eighteenDash, eighteenDash))
    for param in params:
        logq, logbeta, n = param[1], param[2], param[3]
        svp_l2 = direct_svp.svp_attack_via_l2(n=n, log2_bound=logbeta, log2_q=logq)
        svp_loo = direct_svp.svp_attack_via_linf(n=n, log2_bound=logbeta, log2_q=logq)
        bkz_with_prob = estimate_bkz2(log2_bound=logbeta, log_q=logq, n=n)
        cpw_cost = cpw_bit_security(log2_beta=logbeta, log2_q=logq, n=n, m_0=2**21)
        numlimbs = math.ceil(256 / logbeta)
        prod = numlimbs*n
        print("|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|{:<18}|".format(logq, logbeta, n, round(svp_l2, ndigits=2), round(svp_loo, ndigits=2), round(bkz_with_prob, ndigits=2), round(cpw_cost, ndigits=2), numlimbs, round(prod, ndigits=2)))

#estimate_set_of_params(old_params)
#estimate_set_of_params(new_params)
print()
print("===================="*10)
print("old - params")
print()
estimate_set_of_params(old_params)
print()
print("===================="*10)
print("current - params")
print()
estimate_set_of_params(cur_params)
print()
print("===================="*10)
print("future - params")
print()
estimate_set_of_params(hundreds)


old - params

|$log_2(q)$        |$log_2(beta)$     |$n$               |SVP (L2)          |SVP (Loo)         |BKZ attack        |CPW attack        |#limbs            |pi                |
|------------------|------------------|------------------|------------------|------------------|------------------|------------------|------------------|------------------|
|64                |5                 |32                |136.12            |257.92            |135.22            |143.0             |52                |1664              |
|64                |7                 |64                |187.51            |396.8             |160.64            |287.07            |37                |2368              |
|64                |11                |128               |233.94            |476.16            |151.97            |598.14            |24                |3072              |
|64                |16                |256               |315.41            |634.88            |158.74            |1272.